In [ ]:
"""
Using tensorflow 1.15.2
"""
%tensorflow_version 1.x

In [ ]:
import tensorflow as tf 
print(tf.__version__)
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

1.15.2
Found GPU at: /device:GPU:0


In [ ]:
"""
Using a Virtual GPU provided by Colab --> Tesla T4
"""

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5971529699143551978, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 17673446176178668636
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 4225322642779678856
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14912199066
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 14342415585498488147
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [ ]:
"""
Mouting the Google Drive to access the dataset
"""

from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
"""
Specifying device path for Test set
"""

#Please mention the test set path here, below!
test_path = ""

In [ ]:
"""
Importing all the necessary libraries required to import/train/test our Model
"""

import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout

In [ ]:
"""
Using a Transfer Learning Approach to train the model for better accuracy as the dataset available
at hand is less and wont be sufficient enough to obtain a better accuracy by manual building of models.

We are using the VGG16 (Visual Geometry Group 16/OxfordNet) as our TL Model, thus loading thier pre-trained
weights by passing --> weights = 'imagenets'
"""

from keras.applications.vgg16 import VGG16
vgg_model = VGG16(weights = 'imagenet')
vgg_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
"""
Curating a new model as Sequential and removing the last three layers of VGG16 which are:
1) Fully Connected 1 (Dense)
2) Fully Connected 2 (Dense)
3) Predictions (Dense)
"""

model = Sequential()
for layer in vgg_model.layers[:-3]:
  model.add(layer)

In [ ]:
"""
Gives the summary of configuration of Model
"""

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [ ]:
"""
Not training the VGG16 Model, thus setting the layers - trainable parameter as False
"""

for layer in model.layers: 
  layer.trainable = False

In [ ]:
"""
--> Adding a Dense Layer with 1024 units 
--> L2 regularizer has been used to avoid over fluctuation of function co-efficients
--> Dropout of 0.25 is used to remove 25% of the neurons per iteration (avoiding overfitting)
""" 

from keras.regularizers import l2 
model.add(Dense(units = 1024, activation = 'relu', kernel_regularizer = l2(0.01)))
model.add(Dropout(0.25))
model.add(Dense(units = 1, activation = 'sigmoid'))

In [ ]:
"""
Curated Model Summary
Post adding our new Dense Layer (1024 units) the total number of trainable params now become = 25,692,161
"""

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [ ]:
"""
Loading the Pre-trained Model Weights
"""

#Please specify the path for the Pre-trained weights to load them here, below!
model.load_weights("")

In [ ]:
"""
Compiling the model:
Optimizer --> Adam
binary_crossentropy --> Binary crossentropy is a loss function that is used in binary classification tasks. 
These are tasks that answer a question with only two choices (0/1, Yes/No)
"""

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
"""
--> ImageDataGenerator has been imported to transform the incoming images and normalize them
--> The flow_from_directory() method allows you to read the images directly from the
    directory and augment them while the neural network model is learning on the training data
"""

from keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1./255)

test_set = test_datagen.flow_from_directory(test_path, target_size=(224, 224), batch_size=16, class_mode='binary', shuffle=False)

Found 132 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


In [ ]:
"""
Importing Confusion Matrix from sklearn.metrics to test the Model Performance
"""

import numpy as np
from sklearn.metrics import plot_confusion_matrix, confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
"""
Our trained model is predicting over the test_set which has,
5 NG & 5 OK Engine Images using the .predict_generator function
"""

Y_pred = model.predict_generator(test_set)
Y_pred.shape

(10, 1)

In [ ]:
"""
Converting all the predicted values in binary format where,
0 --> NG
1 --> OK
by setting a threshold of 0.5 below which it is deemed NG and above which it is deemed OK
"""

y_pred = np.where(Y_pred < 0.5, 0, 1)
y_pred.shape

(10, 1)

In [ ]:
"""
As one can notice, the model is perfectly performing on the test data, viz
Predicted Not Good Engine = 5 = Absolute Not Good Engine
Predicted OK Engine = 5 = Absolute OK Engine
"""

print('Confusion Matrix')
print(confusion_matrix(test_set.classes, y_pred))

Confusion Matrix
[[5 0]
 [0 5]]


In [ ]:
"""
As the CLassification report claims,
the Model Accuracy is 100% both on Ng & Ok image dataset
"""

print('Classification Report')
target_names = ['Not Good', 'Good']
print(classification_report(test_set.classes, y_pred, target_names=target_names))

Classification Report
              precision    recall  f1-score   support

    Not Good       1.00      1.00      1.00         5
        Good       1.00      1.00      1.00         5

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10

